#### imports

In [1]:
#processing
import os
import re
import numpy as np
import matplotlib.pyplot as plt

#utilities
from utils import *
from text_utils import *
from image_utils import *

#tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.layers import TextVectorization

#VGG16
from tensorflow.keras.applications.vgg16 import VGG16 , preprocess_input , decode_predictions
from tensorflow.keras.preprocessing.image import load_img , img_to_array

#keras
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator



import pickle

In [2]:
seed = 111
np.random.seed(seed)
tf.random.set_seed(seed)

#### import the data

In [3]:
# Path to the images
IMAGES_PATH = "dataset/Flicker/Flicker8k_Dataset"

# Path to the captions
CAPTIONS_PATH = "dataset/Flicker/Flickr8k_text/Flickr8k.token.txt"

# Desired image dimensions
IMAGE_SIZE = (224, 224)

# Vocabulary size
VOCAB_SIZE = 10000

# Max length allowed for any sequence
SEQ_LENGTH = 25

# Dimension for the image embeddings and token embeddings
EMBED_DIM = 512

# Per-layer units in the feed-forward network
FF_DIM = 512

# Other training parameters
BATCH_SIZE = 64
EPOCHS = 30
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
# Load the dataset
captions_mapping, text_data = load_captions_data(CAPTIONS_PATH,SEQ_LENGTH,IMAGES_PATH)

# Split the dataset into training and validation sets
train_data, valid_data = train_val_split(captions_mapping)
print("Number of training samples: ", len(train_data))
print("Number of validation samples: ", len(valid_data))

Number of training samples:  6114
Number of validation samples:  1529


In [7]:
vectorization, image_augmentation = text_vectorization(VOCAB_SIZE, SEQ_LENGTH, text_data)
process_input(img_path, captions, vectorization, IMAGE_SIZE)

<function text_utils.process_input(img_path, captions, vectorization)>

In [16]:

#preprocess_text(VOCAB_SIZE, SEQ_LENGTH, text_data, AUTOTUNE, train_data, valid_data, BATCH_SIZE)
#print(text_data)

['<start> A child in a pink dress is climbing up a set of stairs in an entry way . <end>', '<start> A girl going into a wooden building . <end>', '<start> A little girl climbing into a wooden playhouse . <end>', '<start> A little girl climbing the stairs to her playhouse . <end>', '<start> A little girl in a pink dress going into a wooden cabin . <end>', '<start> A black dog and a spotted dog are fighting <end>', '<start> A black dog and a tri-colored dog playing with each other on the road . <end>', '<start> A black dog and a white dog with brown spots are staring at each other in the street . <end>', '<start> Two dogs of different breeds looking at each other on the road . <end>', '<start> Two dogs on pavement moving toward each other . <end>', '<start> A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl . <end>', '<start> A little girl is sitting in front of a large painted rainbow . <end>', '<start> A small girl in the grass plays with fingerp

#### preprocces the data

### create the CNN model

In [7]:
#import the VGG16 net, and remove the top layers(fully connected layers)
#weight can be a pre-trained model
feature_extracter = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

58900480/58889256 [==============================] - 4s 0us/step


In [9]:
#freeze all layers 
for layer in feature_extracter.layers:
    layer.trainable = False

In [10]:
feature_extracter.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#model has 

### train and save the model